# Task 4 SQL
- Sarah DeConink
- 06/08/2023

## Section 0: SETUP

In [99]:
import sqlite3

# create a connection to the database 
connection = sqlite3.connect('books.db')


In [100]:
import pandas as pd

# display the author's table
pd.options.display.max_columns = 10
pd.read_sql('SELECT * FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [101]:
# display the title's table
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


In [102]:
# display the first five rows of the author_ISBN table
df = pd.read_sql('SELECT * FROM author_ISBN', connection)
df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


## Section 1: SELECT

In [103]:
# Retrieve the 'first' and 'last' names from the author's table
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


## Section 2: WHERE

In [104]:
# Query 1 - Select title, edition, and copyright for all books with copyright years greater than 2016
pd.read_sql("""SELECT title, edition, copyright FROM titles WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


In [105]:
# Query 2 - Locate all the authors whose last name starts with the letter D
pd.read_sql("""SELECT id, first, last FROM authors WHERE last LIKE 'D%'""", connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


In [106]:
# Query 3 - Select the rows of all the authors whose last names start with any character, followed by the letter b, followed by the number of additional characters
pd.read_sql("""SELECT id, first, last FROM authors WHERE first LIKE '_b%'""", connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


## Section 3: ORDER BY

In [107]:
# Query 1 - Sort the titles in ascending order
pd.read_sql('SELECT title FROM titles ORDER BY title ASC', connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In [108]:
# Query 2 - Sort the authors' names by last name, then by first name for authors who have the same last name
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last, first""", connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [109]:
# Query 3 - Sort the authors in descending order by last name and ascending order by first name for authors who have the same last name
pd.read_sql("""SELECT id, first, last FROM authors ORDER BY last DESC, first ASC""", connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


In [110]:
# Query 4 - Get the isbn, title, edition, and copyright where the titles end with 'How to Program' then sort them ascending by title
pd.read_sql("""SELECT isbn, title, edition, copyright FROM titles WHERE title LIKE '%How to Program' ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


## Section 4: INNER JOIN

In [111]:
# Query 1 - Produce a list of authors accompanied by the ISBNs for books written by each author (first five rows)
pd.read_sql("""SELECT first, last, isbn FROM authors INNER JOIN author_ISBN ON authors.id = author_ISBN.id ORDER BY last, first""", connection).head()


,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


## Section 5: INSERT INTO

In [112]:
# Call the Connection's cursor method
cursor = connection.cursor()

In [113]:
# Query 1 and 2 - Insert a new author into the author's table and select it from the table
cursor = cursor.execute("""INSERT INTO authors (first, last) VALUES ('Sue', 'Red')""")

pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])


,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


## Section 6: UPDATE

In [114]:
# Query 1 - Update to change the last name from 'Red' to 'Black'
cursor = cursor.execute("""UPDATE authors SET last='Black' WHERE last='Red' AND first='Sue'""")

# Query 2 - Confirm the update
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])


,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


## Section 7: DELETE FROM

In [115]:
# Query 1 - Delete Sue Black from the authors table using her author name
# We do this instead of the id number because multiple inserts may have been made
cursor = cursor.execute("""DELETE FROM authors WHERE last='Black' AND first='Sue'""")

cursor.rowcount

1

In [116]:
# Query 2 - Read the current author's table
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])


,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


## Section 8: SELECT

## Final 3 Tables of Results

### Table 1 - First 3 Descending Order Titles

In [117]:
# Select the titles table with the edition numbers in descending order (first 3 results)
pd.read_sql("""SELECT title, edition FROM titles ORDER BY edition DESC""", connection).head(3)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8


### Table 2 - Authors Whose First Name Starts with 'A'

In [118]:
# Select from author's table
pd.read_sql("""SELECT * FROM authors WHERE first LIKE 'A%'""", connection)

,id,first,last
0,3,Abbey,Deitel
1,5,Alexander,Wald


### Table 3 - Titles that do NOT end with 'How to Program'

In [119]:
# Select from titles table
pd.read_sql("""SELECT isbn, title, edition, copyright FROM titles WHERE title NOT LIKE '%How to Program' ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134289366,Android 6 for Programmers,3,2016
1,0135404673,Intro to Python for CS and DS,1,2020


In [120]:
connection.close()